<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla y Ignacio Meza
- Auxiliar: Sebastián Tinoco
- Ayudante: Felipe Arias y Diego Cortez

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Angelo Muñoz
- Juan Quilapi


### **Link de repositorio de GitHub:** `https://github.com/FrinyildV/Entregas_MDS_7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [1]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones. 

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.pickle`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except: 
    print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [2]:
df_players = pd.read_csv('stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


In [4]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17588 entries, 0 to 17587
Data columns (total 39 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Name               17588 non-null  object
 1   Nationality        17588 non-null  object
 2   National_Position  1075 non-null   object
 3   Club_Position      17587 non-null  object
 4   Height             17588 non-null  int64 
 5   Weight             17588 non-null  int64 
 6   Preffered_Foot     17588 non-null  object
 7   Age                17588 non-null  int64 
 8   Work_Rate          17588 non-null  object
 9   Weak_foot          17588 non-null  int64 
 10  Skill_Moves        17588 non-null  int64 
 11  Ball_Control       17588 non-null  int64 
 12  Dribbling          17588 non-null  int64 
 13  Marking            17588 non-null  int64 
 14  Sliding_Tackle     17588 non-null  int64 
 15  Standing_Tackle    17588 non-null  int64 
 16  Aggression         17588 non-null  int64

In [30]:
df_players.dtypes[df_players.dtypes==object]

Name                 object
Nationality          object
National_Position    object
Club_Position        object
Preffered_Foot       object
Work_Rate            object
dtype: object

In [33]:
df_players.dtypes[df_players.dtypes!=object]

Height               int64
Weight               int64
Age                  int64
Weak_foot            int64
Skill_Moves          int64
Ball_Control         int64
Dribbling            int64
Marking              int64
Sliding_Tackle       int64
Standing_Tackle      int64
Aggression           int64
Reactions            int64
Interceptions        int64
Vision               int64
Composure            int64
Crossing             int64
Short_Pass           int64
Long_Pass            int64
Acceleration         int64
Speed                int64
Stamina              int64
Strength             int64
Balance              int64
Agility              int64
Jumping              int64
Heading              int64
Shot_Power           int64
Finishing            int64
Long_Shots           int64
Curve                int64
Freekick_Accuracy    int64
Penalties            int64
Volleys              int64
seleccionado         int32
dtype: object

In [14]:
df_players.isna().sum()

Name                     0
Nationality              0
National_Position    16513
Club_Position            1
Height                   0
Weight                   0
Preffered_Foot           0
Age                      0
Work_Rate                0
Weak_foot                0
Skill_Moves              0
Ball_Control             0
Dribbling                0
Marking                  0
Sliding_Tackle           0
Standing_Tackle          0
Aggression               0
Reactions                0
Interceptions            0
Vision                   0
Composure                0
Crossing                 0
Short_Pass               0
Long_Pass                0
Acceleration             0
Speed                    0
Stamina                  0
Strength                 0
Balance                  0
Agility                  0
Jumping                  0
Heading                  0
Shot_Power               0
Finishing                0
Long_Shots               0
Curve                    0
Freekick_Accuracy        0
P

In [3]:
df_salarios = pd.read_csv("salarios.csv")
df_salarios

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


In [12]:
df_salarios.info(),df_salarios.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2534 entries, 0 to 2533
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2534 non-null   int64  
 1   Player         2534 non-null   object 
 2   Weekly Salary  2534 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 59.5+ KB


(None,
 Unnamed: 0       0
 Player           0
 Weekly Salary    0
 dtype: int64)

## 1.1 Predicción de Seleccionados Nacionales 

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego. 

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país. 

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [27]:
# Creando label, si es NaN = > no seleccionado = > 0
# Si tiene alguna posición => seleccionado = > 1 
df_players['seleccionado']= df_players['National_Position'].notnull().astype(int)
df_players[["seleccionado","National_Position"]].sample(5,random_state=1)


,seleccionado,National_Position
12005,0,NaN
5637,0,NaN
3071,1,RCB
9705,0,NaN
11224,0,NaN


In [28]:
df_players[["seleccionado","National_Position"]].value_counts()

seleccionado  National_Position
1             Sub                  556
              LCB                   48
              GK                    47
              RCB                   46
              LB                    39
              RB                    38
              RM                    34
              LM                    32
              ST                    30
              RCM                   25
              LCM                   25
              CAM                   19
              LDM                   19
              LS                    18
              RDM                   18
              RS                    18
              CB                     9
              CM                     9
              CDM                    9
              RW                     7
              LW                     7
              LWB                    4
              RAM                    4
              LAM                    4
              RWB               

In [29]:
df_players[["seleccionado"]].value_counts()

seleccionado
0               16513
1                1075
dtype: int64

Por tanto, hay 16513 jugadores que no son considerados como seleccionados y 1075 que si lo son

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores. 

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [X] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [X] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [X] Crear 3 ``pipelines`` con diferentes clasificadores.
- [X] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [X] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

**Respuesta:**

In [132]:
categoricas = [
    # 'Name', #no importa para entrenamiento
'Nationality',
#'National_Position', #idem a label
'Club_Position',
'Preffered_Foot',
'Weak_foot', # no aparecen como object
'Skill_Moves', # no aparecen como object
'Work_Rate']

#aquellas que no son standard pero son numericas
minmax = [                 
'Marking', #bimodal
'Sliding_Tackle',#bimodal
'Standing_Tackle',#bimodal
'Interceptions',
'Finishing',
'Long_Shots',
'Curve', #?
'Freekick_Accuracy', 
'Volleys',
]

# tienen distribucion "normal" o similar
standard = [ 'Height',
'Weight',
'Age', # log
'Ball_Control',
'Dribbling',
'Aggression',
'Reactions',
'Vision',
'Composure',
'Crossing', #puede ser otra
'Short_Pass',
'Long_Pass',
'Acceleration',
'Speed',
'Stamina',
'Strength',
'Balance',
'Agility',
'Jumping',
'Heading', # ?
'Shot_Power',
'Penalties',# ?
] 

Se utiliza onehotencoder sobre las variables categoricas, StandardScaler sobre las variables numericas con distribución normal o similar y el resto de las variables numericas con alguna distribución distinta se maneja con MinMaxScaller.


In [76]:
prepros = ColumnTransformer(
    [( "OneHot",
      OneHotEncoder(handle_unknown='ignore',sparse=False)
      ,categoricas),
    ('MinMax',
       MinMaxScaler(),
       minmax),
    ('standard',
     StandardScaler(),
     standard
    )
    ],
    remainder='passthrough'
)

Respecto a los clasificadores, siguiendo la metodologia indicada para la selección de modelos:

* Dado que son menos de 100k, se parten los pipelines usando un SVM lineal, luego un KNeighbors y seguido por un SVM no lineal. 

In [77]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
pipe_svm_lineal = Pipeline(
    steps =[
       ("prepros",prepros) ,
       ("svm_lin",SVC(kernel='linear',random_state=15))
    ] 
)
pipe_knn = Pipeline(
    steps =[
       ("prepros",prepros) ,
       ("knn",KNeighborsClassifier())
    ] 
)
pipe_svm_rbf = Pipeline(
    steps =[
       ("prepros",prepros) ,
       ("svm_lin",SVC(kernel='rbf',random_state=15))
    ] 
)

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [X] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [78]:
X=df_players.drop(columns=["seleccionado","Name","National_Position"])
y=df_players["seleccionado"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
 test_size=0.3, stratify=y, random_state=15)

In [79]:
y.value_counts(),y_train.value_counts(),y_test.value_counts()


(0    16513
 1     1075
 Name: seleccionado, dtype: int64,
 0    11559
 1      752
 Name: seleccionado, dtype: int64,
 0    4954
 1     323
 Name: seleccionado, dtype: int64)

In [80]:
train_pipe_svm_lineal=pipe_svm_lineal.fit(X=X_train,y=y_train)
y_pred_1=train_pipe_svm_lineal.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_pred_1))


              precision    recall  f1-score   support

           0       0.95      0.99      0.97      4954
           1       0.75      0.24      0.37       323

    accuracy                           0.95      5277
   macro avg       0.85      0.62      0.67      5277
weighted avg       0.94      0.95      0.94      5277



In [81]:
train_pipe_knn = pipe_knn.fit(X=X_train,y=y_train) 
y_pred_2=train_pipe_knn.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_pred_2))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      4954
           1       0.62      0.11      0.19       323

    accuracy                           0.94      5277
   macro avg       0.78      0.55      0.58      5277
weighted avg       0.93      0.94      0.92      5277



In [82]:
train_pipe_svm_rbf = pipe_svm_rbf.fit(X=X_train,y=y_train)
y_pred_3=train_pipe_svm_rbf.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_pred_3))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       1.00      0.01      0.01       323

    accuracy                           0.94      5277
   macro avg       0.97      0.50      0.49      5277
weighted avg       0.94      0.94      0.91      5277



**Respuesta**:      
De los resultados obtenidos, el modelo con mejor rendimiento fue el del SVM con kernel lineal, obteniendo un f1 macro (macro avg) de 0.67, metrica importante al evaluar datos desbalanceados, en las metricas micro tambien destaca el mismo modelo para precision, recall y f1 consistentemente. 

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?      
**R:**      
Estas metricas indican que tan bueno es el modelo para clasificar esta clase, siendo mejores mientras mas cercanas a 1 esten. 
- ¿Podrían mejorarse los resultados?, ¿Cómo?     
**R:**     
Si, realizando balance de clases, buscando otros modelos de clasificación, optimizar los hiperparámetros de los mismos modelos.
- ¿Influye la cantidad de ejemplos por clase?       
**R:**        
Si, sobre todo cuando las clases están tan desbalanceadas como lo es el problema actual, lo cual genera dificultades al modelo para clasificar la clase con menos elementos, afectando a su vez la capacidad de generalización y produciendo metricas sesgadas.  

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas. 

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características. 

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF'] 
central_ataque = ['RW', 'CAM', 'LW'] 
central = ['RM', 'CM', 'LM'] 
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [X] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [X] Cuente cuantos por clase quedan.
- [X] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [X] Comente los resultados obtenidos.

**Respuesta:**

In [88]:
df_players["Club_Position"].value_counts()

Sub    7492
Res    3146
RCB     633
GK      632
LCB     631
LB      549
RB      548
ST      426
LM      414
RM      414
LCM     354
RCM     353
CAM     315
RDM     266
LDM     266
RS      206
LS      205
LW      133
RW      133
CDM     119
CB       83
CM       79
RWB      45
LWB      45
LAM      36
RAM      36
LF       12
RF       12
CF        4
Name: Club_Position, dtype: int64

In [89]:
df_players_c=df_players.copy()

In [90]:
map= {
    'ST': 'ataque','CF': 'ataque',
    'RW': 'central_ataque','CAM': 'central_ataque',
    'LW': 'central_ataque','RW': 'central_ataque',
    'RM': 'central','CM': 'central','LM': 'central',
    'RWB': 'central_defensa','CDM': 'central_defensa','LWB': 'central_defensa',
    'RB': 'defensa','CB': 'defensa','LB': 'defensa',
    'GK': 'arquero',
}
df_players_c["posicion_general"] = df_players_c["Club_Position"].map(map)

In [93]:
df_players_c["posicion_general"].isna().sum() #la mayoria no tiene posicion según la deficion previa
df_players_c=df_players_c[~df_players_c["posicion_general"].isna()]

In [113]:
df_players_c[["posicion_general"]].value_counts(),df_players_c.shape

(posicion_general
 defensa             1180
 central              907
 arquero              632
 central_ataque       581
 ataque               430
 central_defensa      209
 dtype: int64,
 (3939, 41))

Para el entrenamiento de los modelos, dado que el problema solo cambia la cantidad de clases a predecir y la cantidad de elementos disponibles se mantendran los mismos pipelines y metodologias.

In [104]:
X=df_players_c.drop(columns=["seleccionado","Name","National_Position","posicion_general","Club_Position"])
y=df_players_c["posicion_general"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
 test_size=0.3, stratify=y, random_state=15)

In [100]:
y.value_counts(normalize=True),y_train.value_counts(normalize=True),y_test.value_counts(normalize=True)

(defensa            0.299568
 central            0.230261
 arquero            0.160447
 central_ataque     0.147499
 ataque             0.109165
 central_defensa    0.053059
 Name: posicion_general, dtype: float64,
 defensa            0.299601
 central            0.230323
 arquero            0.160319
 central_ataque     0.147624
 ataque             0.109177
 central_defensa    0.052956
 Name: posicion_general, dtype: float64,
 defensa            0.299492
 central            0.230118
 arquero            0.160745
 central_ataque     0.147208
 ataque             0.109137
 central_defensa    0.053299
 Name: posicion_general, dtype: float64)

Se vuelve a definir el prepros dado que "Club_Position" estaba como columna a preprocesar y se debe ignorar ya que esta directamente correlacionada con el label a predecir

In [107]:
prepros = ColumnTransformer(
    [( "OneHot",
      OneHotEncoder(handle_unknown='ignore',sparse=False)
      ,categoricas.copy().remove("Club_Position")),
    ('MinMax',
       MinMaxScaler(),
       minmax),
    ('standard',
     StandardScaler(),
     standard
    )
    ],
    remainder='passthrough'
)

In [108]:
train_pipe_svm_lineal=pipe_svm_lineal.fit(X=X_train,y=y_train)
y_pred_1=train_pipe_svm_lineal.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_pred_1))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.77      0.81      0.79       129
        central       0.56      0.61      0.58       272
 central_ataque       0.50      0.40      0.45       174
central_defensa       0.50      0.14      0.22        63
        defensa       0.82      0.92      0.87       354

       accuracy                           0.73      1182
      macro avg       0.69      0.65      0.65      1182
   weighted avg       0.72      0.73      0.72      1182



In [109]:
train_pipe_knn = pipe_knn.fit(X=X_train,y=y_train) 
y_pred_2=train_pipe_knn.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_pred_2))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.72      0.71      0.72       129
        central       0.46      0.58      0.51       272
 central_ataque       0.38      0.21      0.27       174
central_defensa       0.44      0.17      0.25        63
        defensa       0.77      0.87      0.81       354

       accuracy                           0.67      1182
      macro avg       0.63      0.59      0.59      1182
   weighted avg       0.65      0.67      0.65      1182



In [110]:
train_pipe_svm_rbf = pipe_svm_rbf.fit(X=X_train,y=y_train)
y_pred_3=train_pipe_svm_rbf.predict(X_test)
print(classification_report(y_true=y_test,y_pred=y_pred_3))

                 precision    recall  f1-score   support

        arquero       1.00      1.00      1.00       190
         ataque       0.80      0.79      0.79       129
        central       0.53      0.66      0.59       272
 central_ataque       0.53      0.26      0.35       174
central_defensa       0.53      0.13      0.21        63
        defensa       0.79      0.95      0.87       354

       accuracy                           0.73      1182
      macro avg       0.70      0.63      0.63      1182
   weighted avg       0.71      0.73      0.71      1182



**Respuesta**:
De los 3 modelos estudiados, el KNN está por debajo en todas las metricas de los otros modelos.    

Respecto a los 2 modelos de SVM estudiados, mirando las metricas en macro, el SVM con kernel RBF tiene mejor precision que el SVM con kernel lineal, pero el SVM con kernel lineal tiene mejores recall y F1.

En particular a los 3 modelos les fue facil distinguir a los jugadores que serían buenos arqueros, dado que es una posición que es muy diferente a las otras. Tambien son buenos distinguiendo las clases "ataque" y "defensa" por si solos. Para las demás posiciones tiende a tener peores metricas. En particular se destaca el caso de las bajas metricas obtenidas para los "central_defensa" en todos los modelos, se le asocia a los pocos ejemplos disponibles de esta clase. 

Se pueden mejorar los resultados siguiendo las mismas pautas anteriores, realizando balance, optimizando hiperparámetros y probando nuevos modelos. Tambien se propone la busqueda de nuevos datos dado que tras la nueva categorizacion y eliminación de ejemplos sin categoria el numero de ejemplos total bajo de ~ 17000 a ~ 4000 ejemplos. 


## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [X] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

EL $R^2$, tambien conocido como coeficiente de determinación, mide estadisticamente que tan bien se ajustan los datos a una linea de regresion, sigue la ecuación:
$R^2=\frac{\sum_{t=1}^T\left(\hat{Y}_t-\bar{Y}\right)^2}{\sum_{t=1}^T\left(Y_t-\bar{Y}\right)^2}$
Donde: 
* $\bar{Y}$ es el promedio de la variable a predecir
* $\hat{Y}_t$ es la prediccion en cada punto
* $Y_t$ es el valor real en cada punto 

Y su valor varia entre 0 y 1, siendo el 1 un ajuste perfecto de la variable predicha con la variable real.

Vamos a filtrar el df_players segun los nombres que tengan un sueldo en el df_salarios

In [115]:
df_salarios.rename(columns={'Player':'Name'},inplace=True)

In [121]:
df_r=pd.merge(
    left=df_salarios,
    right=df_players,
    on="Name",
    how="inner",
    sort=True,
)
df_r.drop(columns='Unnamed: 0',inplace=True)
df_r

,Name,Weekly Salary,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,...,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys,seleccionado
0,Aaron Cresswell,65966.0,England,NaN,LB,170,66,Left,27,High / Medium,...,85,58,71,53,62,75,69,59,45,0
1,Aaron Lennon,46177.0,England,NaN,Sub,165,63,Right,29,High / Medium,...,71,28,64,69,57,63,55,62,68,0
2,Aaron Leya Iseka,13006.0,Belgium,NaN,Sub,182,72,Right,19,Medium / Medium,...,74,59,74,70,66,52,39,63,59,0
3,Aaron Mooy,65966.0,Australia,RF,LDM,174,68,Right,26,Medium / Medium,...,72,46,70,69,72,73,78,73,65,1
4,Aaron Ramsey,158320.0,Wales,LCM,Sub,183,76,Right,26,High / High,...,65,58,81,73,75,70,70,75,79,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1861,Éver Banega,140063.0,Argentina,CAM,CAM,174,71,Right,28,Medium / Medium,...,59,45,74,65,77,83,81,67,65,1
1862,Óscar Duarte,14137.0,Costa Rica,NaN,Sub,186,81,Right,27,Medium / Medium,...,80,75,47,26,24,28,32,37,32,0
1863,Óscar Plano,14202.0,Spain,NaN,RM,180,72,Left,26,High / Low,...,59,46,81,75,72,79,68,66,54,0
1864,Óscar Trejo,23576.0,Argentina,NaN,CAM,180,79,Right,28,Medium / Medium,...,63,66,73,72,68,74,72,70,72,0


Definiendo el pipeline:

In [171]:
prepros = ColumnTransformer(
    [( "OneHot",
      OneHotEncoder(handle_unknown='ignore',sparse=False)
      ,categoricas.copy()),
    ('MinMax',
       MinMaxScaler(),
       minmax),
    ('standard',
     StandardScaler(),
     standard
    )
    ],
    remainder='passthrough'
)

In [215]:
X=df_r.drop(columns=["seleccionado","Name","National_Position","Weekly Salary"])
#eliminar seleccionado o no está en discusion
# por ej un seleccionado de brasil cuesta mucho dinero 
# cosa que no es standard en todos los paises
y=df_r['Weekly Salary']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
 test_size=0.3, random_state=15)

In [272]:
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from xgboost import XGBRegressor 
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lars

from sklearn.metrics import r2_score

pipe_reg = Pipeline(
    steps =[
       ("prepros",prepros) ,
       ("Regresor",XGBRegressor()
) #alpha=2,
    ] 
)

("Regresor",Lars(eps=0.5,n_nonzero_coefs=500)) #alpha=2,

("Regresor",ElasticNet(alpha=0.5,
                              random_state=1,
                              l1_ratio=0.8,
                              warm_start=True,
                              positive=True,
                              selection='random'
                              )) #alpha=2,

In [270]:
train_pipe_reg = pipe_reg.fit(X=X_train,y=y_train) 
y_pred_reg=train_pipe_reg.predict(X_test)
print(r2_score(y_test,y_pred_reg))

0.48814631414258125


**Respuesta**:

Al hacer merge entre los df la perdida de elementos para entrenar fue significativa, por lo que la obtención de nuevos ejemplos puede ser clave para mejorar el resultado, al igual que la selección de hiperparámetros y de caracteristicas.


# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>